In [12]:
import re
import cv2
import easyocr

In [13]:
reader = easyocr.Reader(['en'], gpu=True) # this needs to run only once to load the model into memory

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [11]:
def extract_task_number(text):
    # 괄호 안의 두 숫자를 각각 캡처하는 정규식
    match = re.search(r'\((\d+)/(\d+)\)', text)
    if match:
        task_number = match.group(1)
        last_instruction = match.group(2)
        return task_number, last_instruction
    return None, None

# 테스트 예제
text1 = "TASK : 32-11-46-400-001-A (1/11)"
text2 = "32-11-46-400-001-A (10/11)"

print(extract_task_number(text1))  # 출력: 1
print(extract_task_number(text2))  # 출력: 10


('1', '11')
('10', '11')


In [17]:
text = reader.readtext('frame1.png', detail=0)
# result = extract_task_number(text[1])
# result
not(text)

True

In [4]:
def decode_output(predicted_text, timestamp):
    """OCR 결과가 이전과 다를 경우 변경된 시점 저장"""
    global previous_text, timestamps

    # OCR 결과가 이전과 다를 경우
    if predicted_text != previous_text:
        print(f"[{timestamp} ms] OCR 변경 감지: {predicted_text}")
        timestamps.append((timestamp, predicted_text))  # 변경된 시간 저장
        previous_text = predicted_text  # 이전 텍스트 업데이트

In [ ]:
class SplitVideo():
    def __init__(self, video_path, save_path):
        self.video_path = video_path
        self.save_path = save_path
        self.previous_text = ""
        self.timestamps = []
        
        
    def __call__(self):
        # 비디오 캡처 객체 생성
        cap = cv2.VideoCapture(self.video_path)
        frame_count = 0
        roi = None

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.resize(frame, (1980, 1080))
            
            if roi is None:
                roi = cv2.selectROI("Select ROI", frame, False, False)
                cv2.destroyWindow("Select ROI")
                x, y, w, h = roi

            if frame_count % 6 == 0:
                # 특정 영역 크롭
                cropped_frame = frame[y:y+h, x:x+w]

                # OCR 모델 예측
                inst_number = reader.readtext(cropped_frame, detail=0)
                inst_number = inst_number[1]
                inst_number = inst_number.split("/")[0]
                inst_number = inst_number[-1]

                # 현재 동영상 시간(ms) 가져오기
                timestamp = int(cap.get(cv2.CAP_PROP_POS_MSEC))

                # OCR 결과 비교 및 시간 기록
                self.record_timestemp(inst_number, timestamp)
            
            frame_count += 1
            
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        
        
    def record_timestemp(self, predicted_text, timestamp):
        """OCR 결과가 이전과 다를 경우 변경된 시점 저장"""

        # OCR 결과가 이전과 다를 경우
        if predicted_text != self.previous_text:
            print(f"[{timestamp} ms] OCR 변경 감지: {predicted_text}")
            self.timestamps.append(timestamp, predicted_text)
            self.previous_text = predicted_text  # 이전 텍스트 업데이트

In [8]:
# 비디오 파일 경로
video_path = "Installation of the Lockstay Actuator Hoses.mp4"

# OCR 결과 추적 변수
previous_text = None  # 이전 OCR 결과
timestamps = []  # 변경된 시점 기록

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(video_path)
frame_count = 0
roi = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (1980, 1080))
    
    if roi is None:
        roi = cv2.selectROI("Select ROI", frame, False, False)
        cv2.destroyWindow("Select ROI")
        x, y, w, h = roi

    if frame_count % 6 == 0:
        # 특정 영역 크롭
        cropped_frame = frame[y:y+h, x:x+w]

        # OCR 모델 예측
        inst_number = reader.readtext(cropped_frame, detail=0)
        inst_number = inst_number[1]
        inst_number = inst_number.split("/")[0]
        inst_number = inst_number[-1]

        # 현재 동영상 시간(ms) 가져오기
        timestamp = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # OCR 결과 비교 및 시간 기록
        decode_output(inst_number, timestamp)
    
    frame_count += 1
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 최종 기록된 시간 출력
print("\n=== OCR 변경 시점 기록 ===")
for t, text in timestamps:
    print(f"{t} ms: {text}")

[0 ms] OCR 변경 감지: 1
[22800 ms] OCR 변경 감지: 2
[57900 ms] OCR 변경 감지: 3
[92700 ms] OCR 변경 감지: 4
[106700 ms] OCR 변경 감지: 5
[122700 ms] OCR 변경 감지: 6


KeyboardInterrupt: 